In [28]:
import numpy as np
from tqdm import tqdm
import math
import numba
from numba import prange
import time
from tabulate import tabulate

In [2]:
@numba.jit(nopython=True, parallel=True)
def butterfl_4(arr, inv, n):
    conj = 1j* (-1 if inv else 1)
    for i in prange((n+3)//4):
        j = i*4
        t1, t2 = arr[j], arr[j+1]
        t3 = arr[j+2]
        arr[j] = t1 + t2 + t3 + arr[j + 3]
        arr[j + 1] = t1 - t2 - conj*(t3 - arr[j + 3])
        arr[j + 2] = t1 + t2 - t3 - arr[j + 3]
        arr[j + 3] = t1 - t2 + conj*(t3 - arr[j + 3])

In [3]:
@numba.jit(nopython=True, parallel=True)
def butterfl_2(arr, n):
    for i in prange((n+1)// 2):
        j = i*2
        arr[j], arr[j + 1] = arr[j] + arr[j + 1], arr[j] - arr[j + 1]

In [4]:
@numba.jit(nopython=True)
def rev(num, lg_n):
    res = 0
    for i in range(lg_n):
        if (num & (1<<i)):
            res |= 1<<(lg_n - 1 - i);
    return res

In [5]:
def permut(arr, n, lg_n):
    for i in range(n):
        if (i < rev(i,lg_n)):
            arr[i], arr[rev(i,lg_n)] = arr[rev(i,lg_n)], arr[i]

In [6]:
@numba.jit(nopython=True, parallel=True)
def butterfl_n(arr, n, ln, wlen_pw, max_deg):
    for i in prange((n+ln-1)// ln):
        p = i*ln
        for j in range(ln//2):
            u, v = arr[p + j], arr[p + j + max_deg] * wlen_pw[j]
            arr[p + j] = u + v
            arr[p + j + max_deg] = u - v

In [7]:
def fft(arr, inv, n):
    if (n == 1):
        return

    lg_n = np.ceil(np.log2(n)).astype(int);
    permut(arr, n, lg_n)

    wlen_pw = np.zeros(n, dtype=np.complex128)

    if n >= 4:
        butterfl_4(arr, inv, n)
    elif (n >= 2):
        butterfl_2(arr, n)
        
    ln = 8
    while (ln <= n):
        max_deg = ln >> 1;
        ang = 2 * np.pi / ln * (1 if inv else -1)
        wlen = np.cos(ang) + 1j*np.sin(ang)
        wlen_pw = np.power(wlen, np.arange(0, max_deg), dtype=np.complex128)

        butterfl_n(arr, n, ln, wlen_pw,max_deg)
        
        ln <<= 1
 

    if (inv):
        arr /= n   
    
    return arr

In [22]:
num_thr = [1, 2, 3, 4, 5]
n = [1000, 10000, 100000, 1000000]
time_ex, err = [], []

In [23]:
for n_iter in n:
    for n_thr in num_thr:
        numba.set_num_threads(n_thr)
        a = np.array([i for i in range(n_iter)], dtype=complex)
        two_degr = pow(2, math.ceil(math.log2(len(a))))
        a = np.concatenate([a, np.zeros(two_degr - len(a))])
        
        #start_time1 = time.time()
        ans1 = np.fft.ifft(a, n=two_degr)
        #end_time1 = time.time()

        start_time2 = time.time()
        ans2 = fft(a, True, two_degr)
        end_time2 = time.time()

        time_ex.append(end_time2-start_time2)
        err.append(np.linalg.norm(ans1- ans2))

In [29]:
timex_ex_np = np.array(time_ex).reshape(4, -1)
table = tabulate(np.hstack([np.array(n).reshape(4, 1), timex_ex_np]), num_thr, tablefmt="fancy_grid")

print(table)

╒════════════╤════════════╤════════════╤════════════╤════════════╤════════════╕
│            │          1 │          2 │          3 │          4 │          5 │
╞════════════╪════════════╪════════════╪════════════╪════════════╪════════════╡
│   1000     │ 0.00327229 │ 0.00247002 │ 0.00244761 │ 0.00212216 │ 0.00212908 │
├────────────┼────────────┼────────────┼────────────┼────────────┼────────────┤
│  10000     │ 0.0134025  │ 0.0137198  │ 0.0136051  │ 0.0134852  │ 0.0135539  │
├────────────┼────────────┼────────────┼────────────┼────────────┼────────────┤
│ 100000     │ 0.110117   │ 0.109081   │ 0.109049   │ 0.107486   │ 0.109766   │
├────────────┼────────────┼────────────┼────────────┼────────────┼────────────┤
│      1e+06 │ 0.929432   │ 0.918123   │ 0.92293    │ 0.916224   │ 0.916025   │
╘════════════╧════════════╧════════════╧════════════╧════════════╧════════════╛


In [30]:
err_np = np.array(err).reshape(4, -1)
table = tabulate(np.hstack([np.array(n).reshape(4, 1), err_np]), num_thr, tablefmt="fancy_grid")

print(table)

╒════════════╤═════════════╤═════════════╤═════════════╤═════════════╤═════════════╕
│            │           1 │           2 │           3 │           4 │           5 │
╞════════════╪═════════════╪═════════════╪═════════════╪═════════════╪═════════════╡
│   1000     │ 2.97544e-12 │ 2.97544e-12 │ 2.97544e-12 │ 2.97544e-12 │ 2.97544e-12 │
├────────────┼─────────────┼─────────────┼─────────────┼─────────────┼─────────────┤
│  10000     │ 5.38838e-10 │ 5.38838e-10 │ 5.38838e-10 │ 5.38838e-10 │ 5.38838e-10 │
├────────────┼─────────────┼─────────────┼─────────────┼─────────────┼─────────────┤
│ 100000     │ 5.29746e-08 │ 5.29746e-08 │ 5.29746e-08 │ 5.29746e-08 │ 5.29746e-08 │
├────────────┼─────────────┼─────────────┼─────────────┼─────────────┼─────────────┤
│      1e+06 │ 3.23341e-06 │ 3.23341e-06 │ 3.23341e-06 │ 3.23341e-06 │ 3.23341e-06 │
╘════════════╧═════════════╧═════════════╧═════════════╧═════════════╧═════════════╛
